In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats as st
import math
from tqdm.notebook import tqdm as tqdn
from tqdm import tqdm
import iteration_utilities
import statsmodels
import scipy.optimize
import scipy.spatial.distance as sd
from lmfit import *
from matplotlib.lines import Line2D

mpl.rcParams['font.sans-serif'] = 'Arial'
mpl.rcParams['font.family'] = 'sans-serif'
plt.rcParams['figure.figsize'] = (5, 3)
mpl.rcParams['pdf.fonttype'] = 42
sns.set_style(
    'ticks',
    {
        'xtick.major.size': 4,
        'ytick.major.size': 4,
        'font_color': 'k',
        'axes.edgecolor': 'k',
        'xtick.color': 'k',
        'ytick.color': 'k',
    },
)
sns.set_context('talk', font_scale=1.0)

In [2]:
import os
import altair as alt
alt.data_transformers.enable('json', prefix='tmp/')
alt.themes.enable('vox')



ThemeRegistry.enable('vox')

In [3]:
df = pd.read_csv("../../../fig_3/01_repressor_additivity/pairs_baselinesums.csv")
df.head()

,pair,type,enrichment_ratio_r1_d2,enrichment_ratio_r1_d5,enrichment_ratio_r2_d2,enrichment_ratio_r2_d5,enrichment_fraction_r1_d2,enrichment_fraction_r1_d5,enrichment_fraction_r2_d2,enrichment_fraction_r2_d5,...,d2_med_d5,d2_sd_d5,d2_description,d2_baseline_type,composition,character,act_pair_hit,rep_pair_hit,baseline_sum_d2,baseline_sum_d5
0,Short_nuclear_domain;NCOA2_HUMAN;Nuc_rec_co-ac...,0 control Pair,3.369442,0.365155,3.719474,-0.000660,0.911776,0.562941,0.929442,0.499886,...,2.054285,0.493242,Repressor,Non-hit,A-N,Other,True,False,1.126839,2.462512
1,Silencer_tiles;ENSG00000179833;22 --- Silencer...,0 control Pair,2.414849,-0.758524,3.474812,-0.858615,0.842088,0.371504,0.917477,0.355455,...,0.351655,0.582309,Repressor,Activator,A-A,Activator,True,True,5.959180,0.703310
2,Silencer_tiles;ENSG00000069812;10 --- Silencer...,0 control Pair,-3.045459,-2.324324,-2.057486,-1.610970,0.108037,0.166436,0.193701,0.246635,...,-0.808308,0.670493,Repressor,Repressor,R-R,Repressor,False,True,-3.901673,-1.616616
3,Short_nuclear_domain;ZN473_HUMAN;KRAB;1;41 ---...,0 control Pair,2.151337,-0.643566,3.060479,-0.645404,0.816257,0.390292,0.892962,0.389989,...,0.745666,0.626952,Activator,Activator,A-A,Activator,True,True,5.391484,1.491332
4,Short_nuclear_domain;HERC2_HUMAN;Cyt-b5;1207;7...,0 control Pair,-1.554031,-1.183923,-1.379328,-2.532059,0.254042,0.305629,0.277664,0.147406,...,1.827117,0.364559,Repressor,Non-hit,R-N,Other,False,True,-3.468438,-1.219896


In [4]:
df = df[df['type']=='1 control Pair']
df = df.dropna(subset=['avg_enrichment_d2', 'avg_enrichment_d5'])
df.head()

,pair,type,enrichment_ratio_r1_d2,enrichment_ratio_r1_d5,enrichment_ratio_r2_d2,enrichment_ratio_r2_d5,enrichment_fraction_r1_d2,enrichment_fraction_r1_d5,enrichment_fraction_r2_d2,enrichment_fraction_r2_d5,...,d2_med_d5,d2_sd_d5,d2_description,d2_baseline_type,composition,character,act_pair_hit,rep_pair_hit,baseline_sum_d2,baseline_sum_d5
5059,Silencer_tiles;ENSG00000179833;22 --- Random_c...,1 control Pair,3.746226,0.738330,3.545054,0.308524,0.930648,0.625222,0.921090,0.553260,...,0.834829,1.815016,Control,Control,A-C,Activator,True,False,1.600885,1.186484
5060,DMD_control_tiles;ENSG00000198947;;171; --- Sh...,1 control Pair,2.686799,1.329900,2.385927,0.903866,0.865570,0.715412,0.839404,0.651698,...,0.745666,0.626952,Activator,Activator,C-A,Activator,True,False,1.327140,1.578151
5061,Short_nuclear_domain;NCOA2_HUMAN;Nuc_rec_co-ac...,1 control Pair,3.755808,0.278484,3.969173,0.252153,0.931075,0.548108,0.939982,0.543584,...,-0.060837,1.791538,Control,Control,A-C,Activator,True,False,0.958324,0.347389
5062,Short_nuclear_domain;PYGO1_HUMAN;PHD;330;56 --...,1 control Pair,0.684541,0.567439,1.012714,0.912873,0.616445,0.597081,0.668622,0.653114,...,0.576543,1.684559,Control,Control,A-C,Activator,True,False,-1.865229,2.165385
5063,Random_control;;;205; --- Short_nuclear_domain...,1 control Pair,3.500972,0.572337,3.803942,-0.398164,0.918840,0.597898,0.933186,0.431438,...,0.408226,0.435211,Activator,Activator,C-A,Activator,True,True,0.958324,0.347389


In [5]:
df['avg_enrichment_d5']

5059    0.523427
5060    1.116883
5061    0.265318
5062    0.740156
5063    0.087086
          ...   
7819   -3.796554
7830   -2.313623
7832    1.497986
7837    2.377984
7839    1.859543
Name: avg_enrichment_d5, Length: 2443, dtype: float64

In [6]:
alt.Chart(df).mark_point().encode(
    x="avg_enrichment_d2",
    y="avg_enrichment_d5",
    color="character",
    tooltip = ['d1_Gene', 'd2_Gene']
)

alt.Chart(...)

In [7]:
def get_chart(gene):
    dq = df[(df["d1_Gene"] == gene) | (df["d2_Gene"] == gene)]
    return (
        alt.Chart(dq)
        .mark_point()
        .encode(
            x=alt.X("avg_enrichment_d2", scale=alt.Scale(domain=(-5, 5))),
            y=alt.Y("avg_enrichment_d5", scale=alt.Scale(domain=(-5, 5))),
            color=alt.Color("character", scale=alt.Scale(scheme='dark2')),
            tooltip=["d1_Gene", "d2_Gene"],
        )
    )

In [8]:
get_chart("SMCA2")

alt.Chart(...)

In [9]:
bprime = 0.85
lmbda = 20
gamma = 0.75


def p(m):
    beta = bprime/(1-bprime)
    mfac = scipy.special.factorial(m)
    myfac = scipy.special.factorial(m * np.exp(gamma *))
    pon = bprime * (np.power(lmbda, m) * np.exp(-lmbda))/mfac
    poff = (1-bprime) * (np.exp(-lmbda) * )
    return pon + poff

SyntaxError: invalid syntax (949312341.py, line 9)

In [ ]:
x = np.arange(0, 50)
y = p(x)

fig, ax = plt.subplots()
ax.plot(x, y)

In [ ]:
p

Transition from OFF to ON with rate $c_f$, and from ON to OFF with rate $c_b$
Produce mRNA at rate $k_b$, and degrade at rate $k_d$

$$\begin{align*}
G &= G_0 + G_1 \\
G_\alpha(z,t) &= \sum\limits_{m=0}^{\infty}{z^mP_\alpha(m,t)} \\
\end{align*}$$

To compute $P(m,t)$ we want the coefficient for the $z^m$ term in $G(z,t)$

We then define:
$$\begin{align*}
\Phi(a, b, x) &= ~_1F_1(a; b; x) = \mathtt{scipy.special.hyp1f1(a, b, x)} \\
F_s(t) &= \Phi(-c_f, 1-c_f-c_b, k_b e^{-t}(1-z)) \\
F_{ns}(t) &= -\frac{c_fk_b(1-z)}{(c_f + c_b)(1-c_f-c_b)} e^{-(c_f + c_b)t} \times 
\Phi(c_b, 1+c_f+c_b, k_b e^{-t}(1-z))
\end{align*}$$

By black magic math, we have

$$\begin{align*}
G(z,t) &= F_s(t) \cdot \Phi(c_f, c_f + c_b; -k_b(1-z)) + F_{ns}(t) \Phi(1-c_b, 2-c_f-c_b, -k_b(1-z))\\
\end{align*}$$

For details, see [BHJ09](https://journals.aps.org/pre/pdf/10.1103/PhysRevE.79.031911)

Ok, let's try to write this garbage down in code. 

In [11]:
cf = 85
cb = 10
kb = 2400
kd = 2

def Phi(a, b, x): 
    return scipy.special.hyp1f1(a, b, x)

def Fs(t, z):
    return Phi(-cf, 1-cf-cb, kb*np.exp(-t)*(1-z))

def Fns(t, z):
    t1 = - (cf*kb*(1-z))/((cf+cb)(1-cf-cb)) * np.exp(-(cf+cb)*t)
    t2 = Phi(cb, 1+cf+cb, kb*np.exp(-t)*(1-z))
    return t1 * t2

In [32]:
beta = 85
lmbda = 1500
gamma = 2
tmax = 5


def integrand(t, m, beta, lmbda, gamma):
    return np.exp(-beta * t) * st.poisson(lmbda / gamma).pmf(m * np.exp(-gamma * t))


def get_off_integral(m, t_current, beta, lmbda, gamma):
    i = lambda t: integrand(t, m, beta, lmbda, gamma)
    return scipy.integrate.quad(i, 0, t_current)

In [33]:
x = np.arange(0, 1500, step=10)
y = [get_off_integral(m, 1.25, beta, lmbda, gamma)[] for m in x]

fig, ax = plt.subplots()
# ax.plot(x, y)

TypeError: get_off_integral() takes 5 positional arguments but 6 were given